In [1]:
import torch
import torchtext
import re
import nltk
import itertools

# Implementación del algoritmo word2vec

Link de ayuda: [Word2Vec](https://rguigoures.github.io/word2vec_pytorch/)

**Objetivo.** Nuestro objetivo es utilizar un corpus de texto para entender el significado de los gramas que aparecen en el mismo. Para hacer eso, hay que definir un **modelo de lenguaje**, es decir una serie de reglas que nos van a conducir a entender el texto. 

El algoritmo *word2vec* define dos modelos de lenguaje distintos, pero basados en el hecho de que el significado de las palabras puede deducirse de su contexto (**hipótesis distribucional**). Esto se hace para entender el texto a través de su segmentación en palabras y, posteriormente, de la asignación de un vector de $\mathbb{R}^{n}$ a cada palabra. 

## CBOW

El primer algoritmo *word2vec* propone un modelo de lenguaje de la siguiente manera:

* El corpus de texto con el que se trabaja consiste en una sucesión ordenada de palabras y/o signos de puntuación, tabulación, fin de página, etc., que para generalizar, llamaremos "gramas". 

* Los gramas que aparecen en el texto pertenecen a un vocabulario $V = \{ w_1, w_2, \ldots, w_{|V|} \}$. Este vocabulario contiene a los gramas codificadss con *one-hot* vectors, es decir, 

$$
w_{i_j} = 
\begin{cases} 
  1 & \mbox{si} & j = i \\
  0 & \mbox{si} & j \neq i 
\end{cases}
$$


* Se definen las variables aleatorias $\mathbf{X}_{-m}, \mathbf{X}_{-m+1}, \ldots, \mathbf{X}_{-1}, \mathbf{X}_0, \mathbf{X}_{1}, \ldots, \mathbf{X}_{m}$, todas ellas con realizaciones en $V$, y para cada sucesión contigua de gramas encontrada en el texto, se tiene una probabilidad conjunta 

$$
P(\mathbf{X}_{-m} = x_{-m}, \mathbf{X}_{-m+1} = x_{-m+1}, \ldots, \mathbf{X}_{-1} = x_{-1}, \mathbf{X}_0 = x_{0}, \mathbf{X}_{1} = x_1, \ldots, \mathbf{X}_{m-1} =x_{m-1}, \mathbf{X}_{m} = x_{m})
$$

* Se desea estimar la probabilidad 
$$
P(\mathbf{X}_0 = w_i | \mathbf{X}_{-m} = x_{-m}, \mathbf{X}_{-m+1} = x_{-m+1}, \ldots, \mathbf{X}_{-1} = x_{-1}, \mathbf{X}_{1} = x_1, \ldots, \mathbf{X}_{m-1} =x_{m-1}, \mathbf{X}_{m} = x_{m})
$$
para todo $i=1,\ldots,|V|$ y para cada conjunto posible de $x_{-m},\ldots,x_{m}$ de vectores pertenecientes a $V$.

## Skip-gram

El modelo *Skip-gram* propone algo similar, aunque esta vez se busca estimar

$$
P(\mathbf{X}_{-m} = x_{-m}, \mathbf{X}_{-m+1} = x_{-m+1}, \ldots, \mathbf{X}_{-1} = x_{-1}, \mathbf{X}_{1} = x_1, \ldots, \mathbf{X}_{m-1} =x_{m-1}, \mathbf{X}_{m} = x_{m} | \mathbf{X}_0 = x_0)
$$

Si se asume idependencia condicional, esta probabilidad es igual a

$$
\prod_{i=-m\\i\neq 0}^{m} P(\mathbf{X}_{i} = x_i | \mathbf{X}_0 = x_0)
$$

# Simulación

El corpus de texto *Brown* es un conjunto de archivos de texto divididos por categoría.

In [2]:
nltk.download('brown', download_dir='/home/lestien/anaconda3/envs/TorchEnv/nltk_data')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     /home/lestien/anaconda3/envs/TorchEnv/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
print('Corpus categories:')
print(brown.categories())
print()

Corpus categories:
['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']



Definimos un corpus para cada categoría. Un corpus es un conjunto de frases que aparecen en el conjunto de textos en la categoría. Un corpus es una lista de listas de palabras. Por ejemplo, para la categoría "news" se tiene el siguiente corpus:

In [4]:
categories = ['news']
corpus_unpreproceced = brown.sents(categories=categories)
print('Cantidad de frases en estas categorías: ',len(corpus_unpreproceced))
print()

print('Algunos ejemplos:')
print()
n = 5
for i in range(n):
    print('Frase {}:'.format(i+1))
    print(corpus_unpreproceced[i])
    print()

Cantidad de frases en estas categorías:  4623

Algunos ejemplos:

Frase 1:
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']

Frase 2:
['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']

Frase 3:
['The', 'September-October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate',

In [5]:
corpus = []

print('Procesando texto...')
for sentence in corpus_unpreproceced:
    text = ' '.join(sentence)
    text = text.lower()
    text.replace('\n', ' ')
    text = re.sub('[^a-z ]+', '', text)
    corpus.append([w for w in text.split() if w != ''])
    
print()
print('Nuevo texto:')
print()
n = 5
for i in range(n):
    print('Frase {}:'.format(i+1))
    print(corpus_unpreproceced[i])
    print()

Procesando texto...

Nuevo texto:

Frase 1:
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']

Frase 2:
['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']

Frase 3:
['The', 'September-October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate', 'Ivan', 'Allen', 'Jr.', '.']



Definimos el vocabulario para este corpus:

In [6]:
no_sentence = '<NS>'
vocabulary = set(itertools.chain.from_iterable(corpus))
vocabulary.add(no_sentence)

word_to_index = {w: idx for (idx, w) in enumerate(vocabulary)}
index_to_word = {idx: w for (idx, w) in enumerate(vocabulary)}

Obtenemos las muestras en forma de tupla `(contexto, palabra central)`:

In [8]:
m = 2
samples = []

for sentence in corpus:
    for i, word in enumerate(sentence):
        first_context_word_index = max(0,i-m)
        last_context_word_index = min(i+m+1, len(sentence))
        
        
        context = [no_sentence for j in range(i-m,first_context_word_index)] + sentence[first_context_word_index:i] \
                + sentence[i+1:last_context_word_index] + [no_sentence for j in range(last_context_word_index,i+m+1)]
        
        samples.append((context, word))
        
print('Cantidad de muestras disponibles: ', len(samples))

Cantidad de muestras disponibles:  87004


In [17]:
class BrownDataset(torch.utils.data.Dataset):
    
    def __init__(self, categories, root='./', preprocessing=None, context_size=2):
        nltk.download('brown', download_dir=root)
        from nltk.corpus import brown
        self.corpus_unpreproceced = brown.sents(categories=categories)
        self.preprocessing = preprocessing
        self.context_size = context_size
        
        if self.preprocessing:
            self.corpus = self.preprocessing(self.corpus_unpreproceced)
        else:
            self.corpus = self.corpus_unpreproceced
        
        no_sentence = '<NS>'
        self.vocabulary = set(itertools.chain.from_iterable(self.corpus))
        self.vocabulary.add(no_sentence)

        self.word_to_index = {w: idx for (idx, w) in enumerate(self.vocabulary)}
        self.index_to_word = {idx: w for (idx, w) in enumerate(self.vocabulary)}
        
        samples = []
        for sentence in self.corpus:
            for i, word in enumerate(sentence):
                first_context_word_index = max(0,i-self.context_size)
                last_context_word_index = min(i+self.context_size+1, len(sentence))
                
                context = [no_sentence for j in range(i-self.context_size,first_context_word_index)] + \
                          sentence[first_context_word_index:i] + \
                          sentence[i+1:last_context_word_index] + \
                          [no_sentence for j in range(last_context_word_index,i+self.context_size+1)]
                
                samples.append((context, word))
        
        self.samples = samples
        
    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self, idx):
        
        context, word = self.samples[idx]
        idx_context = torch.empty(len(context), dtype=torch.long)
        idx_word = torch.tensor(self.word_to_index[word], dtype=torch.long)
        for i, w in enumerate(context):
            idx_context[i] = self.word_to_index[w]

        return idx_context, idx_word
       

        
class PreprocessBrown(object):
    
    def __call__(self,corpus_unpreproceced):
        corpus = []
        for sentence in corpus_unpreproceced:
            text = ' '.join(sentence)
            text = text.lower()
            text.replace('\n', ' ')
            text = re.sub('[^a-z ]+', '', text)
            corpus.append([w for w in text.split() if w != ''])
        return corpus


categories = ['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', \
              'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', \
              'reviews', 'romance', 'science_fiction']
context_size = 2
train_dataset = BrownDataset(categories=categories,
                             root='/home/lestien/anaconda3/envs/TorchEnv/nltk_data',
                             preprocessing=PreprocessBrown(),
                             context_size=context_size)

val_dataset = BrownDataset(categories=categories,
                             root='/home/lestien/anaconda3/envs/TorchEnv/nltk_data',
                             preprocessing=PreprocessBrown(),
                             context_size=context_size)

print('Cantidad de muestas:', len(train_dataset))
print()
c, w = train_dataset[0]
print('context:')
for i in c:
    print(train_dataset.index_to_word[i.tolist()])
print()
print('Palabra central:')
print(train_dataset.index_to_word[w.tolist()])

[nltk_data] Downloading package brown to
[nltk_data]     /home/lestien/anaconda3/envs/TorchEnv/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /home/lestien/anaconda3/envs/TorchEnv/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Cantidad de muestas: 1005119

context:
<NS>
<NS>
fulton
county

Palabra central:
the


Ahora falta hacer el resto del modelo.

In [18]:
from torch.utils.data import SubsetRandomSampler, DataLoader

val_size = .02
batch_size = 64

NUM_TRAIN = int((1 - val_size) * len(train_dataset))
NUM_VAL = len(train_dataset) - NUM_TRAIN

train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              sampler=SubsetRandomSampler(range(NUM_TRAIN)))

val_dataloader = DataLoader(val_dataset, 
                            batch_size=batch_size, 
                            sampler=SubsetRandomSampler(range(NUM_TRAIN, NUM_TRAIN+NUM_VAL)))

In [19]:
import torch.optim as optim

def CheckAccuracy(loader, model, device, input_dtype, target_dtype, lm):  
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=input_dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=target_dtype)
            
            if lm == 'CBOW':
                scores = model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
                
            elif lm == 'SkipGram':
                y = y.view(-1,1)
                y = y.expand(-1,x.size()[1])
                scores = model(y)[:,:,0]
                _, preds = scores.max(1)
                for i in range(x.size()[1]):
                    num_correct += (preds == x[:,i]).sum()
                    num_samples += preds.size(0)
        
        return num_correct, num_samples
        

def TrainWord2Vec(model, data, epochs=1, learning_rate=1e-2, sample_loss_every=100, lm='CBOW'):
    
    input_dtype = data['input_dtype'] 
    target_dtype = data['target_dtype']
    device = data['device']
    train_dataloader = data['train_dataloader']
    val_dataloader = data['val_dataloader']
    
    performance_history = {'iter': [], 'loss': [], 'accuracy': []}
    
    model.train()
    model = model.to(device=device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    for e in range(epochs):
        for t, (x,y) in enumerate(train_dataloader):
            x = x.to(device=device, dtype=input_dtype)
            y = y.to(device=device, dtype=target_dtype)

            if lm == 'CBOW':
                scores = model(x) # Forward pass
                loss = model.loss(scores,y) # Backward pass
                
            elif lm == 'SkipGram':
                y = y.view(-1,1)
                y = y.expand(-1,x.size()[1])
                scores = model(y)
                loss = model.loss(scores,x)
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % sample_loss_every == 0:
                num_correct, num_samples = CheckAccuracy(val_dataloader, model, device, input_dtype, target_dtype, lm=lm)
                performance_history['iter'].append(t)
                performance_history['loss'].append(loss.item())
                performance_history['accuracy'].append(float(num_correct) / num_samples)
                print('Epoch: %d, Iteration: %d, Accuracy: %d/%d ' % (e, t, num_correct, num_samples))
                
    num_correct, num_samples = CheckAccuracy(val_dataloader, model, device, input_dtype, target_dtype, lm)
    print('Final accuracy: %.2f%%' % (100 * float(num_correct) / num_samples) )
    
    return performance_history

In [20]:
import torch.nn as nn

class Word2VecCBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super(Word2VecCBOW,self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, context_word):
        emb = self.embeddings(context_word).mean(dim=1)
        return self.linear(emb)
    
    def loss(self, scores, target):
        m = nn.CrossEntropyLoss()
        return m(scores,target)
    
vocab_size = len(train_dataset.vocabulary)
embedding_size = 50
model = Word2VecCBOW(vocab_size, embedding_size)

In [15]:
class Word2VecSkipGram(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super(Word2VecSkipGram,self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, word):
        emb = self.embeddings(word)
        lin = self.linear(emb).permute(0,2,1)
        return lin
    
    def loss(self, scores, target):
        m = nn.CrossEntropyLoss()
        return m(scores,target)
    
vocab_size = len(train_dataset.vocabulary)
embedding_size = 50
model = Word2VecSkipGram(vocab_size, embedding_size)

In [22]:
# Especificaciones de cómo adquirir los datos para entrenamiento:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
data = {
    'device': device,
    'input_dtype': torch.long, 
    'target_dtype': torch.long,
    'train_dataloader': train_dataloader,
    'val_dataloader': val_dataloader
}

# Hiperparámetros del modelo y otros:
epochs = 5 # Cantidad de epochs
sample_loss_every = 200 # Cantidad de iteraciones para calcular la cantidad de aciertos
learning_rate = 1e-2 # Tasa de aprendizaje

# Entrenamiento:
performance_history = TrainWord2Vec(model, data, epochs, learning_rate, sample_loss_every, lm='CBOW')

Epoch: 0, Iteration: 0, Accuracy: 947/20103 
Epoch: 0, Iteration: 200, Accuracy: 1041/20103 
Epoch: 0, Iteration: 400, Accuracy: 1104/20103 
Epoch: 0, Iteration: 600, Accuracy: 1142/20103 
Epoch: 0, Iteration: 800, Accuracy: 1174/20103 
Epoch: 0, Iteration: 1000, Accuracy: 1201/20103 
Epoch: 0, Iteration: 1200, Accuracy: 1208/20103 
Epoch: 0, Iteration: 1400, Accuracy: 1228/20103 
Epoch: 0, Iteration: 1600, Accuracy: 1245/20103 
Epoch: 0, Iteration: 1800, Accuracy: 1252/20103 
Epoch: 0, Iteration: 2000, Accuracy: 1254/20103 
Epoch: 0, Iteration: 2200, Accuracy: 1262/20103 
Epoch: 0, Iteration: 2400, Accuracy: 1272/20103 
Epoch: 0, Iteration: 2600, Accuracy: 1279/20103 
Epoch: 0, Iteration: 2800, Accuracy: 1287/20103 
Epoch: 0, Iteration: 3000, Accuracy: 1296/20103 
Epoch: 0, Iteration: 3200, Accuracy: 1302/20103 
Epoch: 0, Iteration: 3400, Accuracy: 1306/20103 
Epoch: 0, Iteration: 3600, Accuracy: 1309/20103 
Epoch: 0, Iteration: 3800, Accuracy: 1313/20103 
Epoch: 0, Iteration: 4000, A

Epoch: 2, Iteration: 2600, Accuracy: 1434/20103 
Epoch: 2, Iteration: 2800, Accuracy: 1436/20103 
Epoch: 2, Iteration: 3000, Accuracy: 1433/20103 
Epoch: 2, Iteration: 3200, Accuracy: 1435/20103 
Epoch: 2, Iteration: 3400, Accuracy: 1440/20103 
Epoch: 2, Iteration: 3600, Accuracy: 1441/20103 
Epoch: 2, Iteration: 3800, Accuracy: 1444/20103 
Epoch: 2, Iteration: 4000, Accuracy: 1448/20103 
Epoch: 2, Iteration: 4200, Accuracy: 1442/20103 
Epoch: 2, Iteration: 4400, Accuracy: 1441/20103 
Epoch: 2, Iteration: 4600, Accuracy: 1442/20103 
Epoch: 2, Iteration: 4800, Accuracy: 1445/20103 
Epoch: 2, Iteration: 5000, Accuracy: 1449/20103 
Epoch: 2, Iteration: 5200, Accuracy: 1440/20103 
Epoch: 2, Iteration: 5400, Accuracy: 1440/20103 
Epoch: 2, Iteration: 5600, Accuracy: 1442/20103 
Epoch: 2, Iteration: 5800, Accuracy: 1447/20103 
Epoch: 2, Iteration: 6000, Accuracy: 1445/20103 
Epoch: 2, Iteration: 6200, Accuracy: 1445/20103 
Epoch: 2, Iteration: 6400, Accuracy: 1447/20103 
Epoch: 2, Iteration:

Epoch: 4, Iteration: 5200, Accuracy: 1481/20103 
Epoch: 4, Iteration: 5400, Accuracy: 1478/20103 
Epoch: 4, Iteration: 5600, Accuracy: 1479/20103 
Epoch: 4, Iteration: 5800, Accuracy: 1481/20103 
Epoch: 4, Iteration: 6000, Accuracy: 1475/20103 
Epoch: 4, Iteration: 6200, Accuracy: 1473/20103 
Epoch: 4, Iteration: 6400, Accuracy: 1478/20103 
Epoch: 4, Iteration: 6600, Accuracy: 1481/20103 
Epoch: 4, Iteration: 6800, Accuracy: 1483/20103 
Epoch: 4, Iteration: 7000, Accuracy: 1482/20103 
Epoch: 4, Iteration: 7200, Accuracy: 1483/20103 
Epoch: 4, Iteration: 7400, Accuracy: 1482/20103 
Epoch: 4, Iteration: 7600, Accuracy: 1484/20103 
Epoch: 4, Iteration: 7800, Accuracy: 1482/20103 
Epoch: 4, Iteration: 8000, Accuracy: 1484/20103 
Epoch: 4, Iteration: 8200, Accuracy: 1488/20103 
Epoch: 4, Iteration: 8400, Accuracy: 1482/20103 
Epoch: 4, Iteration: 8600, Accuracy: 1484/20103 
Epoch: 4, Iteration: 8800, Accuracy: 1482/20103 
Epoch: 4, Iteration: 9000, Accuracy: 1479/20103 
Epoch: 4, Iteration: